# In this notebook binning is applied on TOF images, the weighted average scheme is tested first, then summing is applied
## each given folder is processed, the user should specify the desired number of bins in the parameter nBins

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import os, fnmatch
from os import listdir
from scipy import ndimage
#%matplotlib inline
%matplotlib notebook

In [2]:
def weightedaverageimage(imgs,size) :
    dims=imgs.shape
    w=np.zeros(imgs.shape)
    M=size**2
    for i in np.arange(dims[2]) :
        f=ndimage.filters.uniform_filter(imgs[:,:,i], size=size)*M
        f2=ndimage.filters.uniform_filter(imgs[:,:,i]**2, size=size)*M
        sigma=np.sqrt(1/(M-1)*(f2-(f**2)/M))
        
        w[:,:,i]=1.0/sigma
        
    wsum=w.sum(axis=2)
    for i in np.arange(dims[2]) :
        w[:,:,i]=w[:,:,i]/wsum
        
    imgs=w*imgs
    img=imgs.sum(axis=2)
    
    return img

In [3]:
def sumimage(imgs):
    summed_img=np.zeros((imgs.shape[0], imgs.shape[1]))
    for i in range(imgs.shape[2]):
        summed_img+=imgs[:,:,i]
    return summed_img

In [5]:
def BinImages(pathdata,folder_output, nBins,filename_output):
    
    if not os.path.exists(folder_output):
        os.makedirs(folder_output)
    
    files = (sorted(fnmatch.filter(listdir(pathdata),'*.fits')))
    nImagesPerBin = int((len(files)-1)/nBins)
    for i in range(0, nBins):
    #     imgs=[]
        for j in range(0,nImagesPerBin):
            names = pathdata + files[i*nImagesPerBin+j]
    #         print(names)
            with fits.open(names) as f:
                if j==0:
                    x_size = np.shape(f[0].data)[0]
                    y_size = np.shape(f[0].data)[1]
    #                 print(x_size, y_size, nImagesPerBin)
                    imgs=np.zeros((x_size,y_size, nImagesPerBin))
    #                 print(imgs.shape)
                imgs[:,:,j] = f[0].data
#         binned_img = weightedaverageimage(imgs,3)
        binned_img = sumimage(imgs)
#         filename_output = 'IMAT_tomo_binned_'
        filename=filename_output+str(i).zfill(5)
        fits.writeto(folder_output+filename+'.fits',binned_img)




In [15]:
path_folders = "/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/"
list_folders = sorted(fnmatch.filter(os.listdir(path_folders),'*_Corrected'))

nBins = 350

display(list_folders)
display(list_folders[0]+'_Binned')

for i in range(0, len(list_folders)):
    folder_output = path_folders+list_folders[i]+'_Binned/'
    folder_input = path_folders+list_folders[i]+'/'
    filename_output = list_folders[i]+'_Binned_'
    print(folder_input, folder_output)
    BinImages(folder_input, folder_output,nBins,filename_output)

['Flat1_Corrected',
 'Sample_10090deg_Corrected',
 'Sample_10099deg_Corrected',
 'Sample_10108deg_Corrected',
 'Sample_10117deg_Corrected',
 'Sample_10126deg_Corrected',
 'Sample_10135deg_Corrected',
 'Sample_10144deg_Corrected',
 'Sample_10153deg_Corrected',
 'Sample_10162deg_Corrected',
 'Sample_10171deg_Corrected',
 'Sample_10180deg_Corrected']

'Flat1_Corrected_Binned'

/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Flat1_Corrected/ /media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Flat1_Corrected_Binned/
/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10090deg_Corrected/ /media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10090deg_Corrected_Binned/
/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10099deg_Corrected/ /media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10099deg_Corrected_Binned/
/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10108deg_Corrected/ /media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10108deg_Corrected_Binned/
/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10117deg_Corrected/ /media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_10117deg_Corrected_Binned/
/media/carminati_c/Data2/IMAT_Nov2018/03_VerticalSetting_3hruns/Sample_1012

In [ ]:
# path_folders = "D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/Sample/"
# list_folders = sorted(os.listdir(path_folders))
# display(list_folders)
path_folders = "D:/Kaestner/ISIS_IAEAData/RB1730044/PSI_Cylinder/Sample/RB1730044/Tomo_test/Sample/"
list_folders = sorted(fnmatch.filter(os.listdir(path_folders),'*_Corrected'))

nBins = 165

# display(list_folders)

for i in range(0, len(list_folders)):
    folder_output = path_folders+list_folders[i]+'_Binned/'
    folder_input = path_folders+list_folders[i]+'/'
    print(folder_input, folder_output)
    BinImages(folder_input, folder_output,nBins)
    